In [1]:
import pandas as pd

In [2]:
result_file = "test_result_w_succ_diff_nodes_gcn__2025_04_03_12_31.csv"

In [3]:
EPS = 1

In [4]:
df = pd.read_csv(result_file)

df = df.rename(columns={"Dataset": "Graph", "Actual": "Rank"})

df.head()

,Graph,Rank,Predicted
0,star_graph_n7,2.0,2.684093
1,star_graph_n7,0.0,0.000000
2,star_graph_n7,0.0,0.000000
3,star_graph_n7,0.0,0.000000
4,star_graph_n7,3.0,3.192155


In [5]:
df['Difference'] = abs(df['Rank']-df['Predicted'])
df['Difference_sq'] = df['Difference']**2
# df.head()

In [6]:
df['EPS'] = df['Rank'].apply(lambda x: EPS if x==0.0 else 0.0)
# df

In [7]:
df['RE'] = df['Difference']/(df['Rank']+df['EPS'])
# df.head()

In [8]:
df

,Graph,Rank,Predicted,Difference,Difference_sq,EPS,RE
0,star_graph_n7,2.0,2.684093,0.684093,0.467983,0.0,0.342046
1,star_graph_n7,0.0,0.000000,0.000000,0.000000,1.0,0.000000
2,star_graph_n7,0.0,0.000000,0.000000,0.000000,1.0,0.000000
3,star_graph_n7,0.0,0.000000,0.000000,0.000000,1.0,0.000000
4,star_graph_n7,3.0,3.192155,0.192155,0.036924,0.0,0.064052
...,...,...,...,...,...,...,...
7938,graph_powerlaw_cluster_graph_n7,1.0,1.039286,0.039286,0.001543,0.0,0.039286
7939,graph_powerlaw_cluster_graph_n7,1.0,0.942022,0.057978,0.003362,0.0,0.057978
7940,graph_powerlaw_cluster_graph_n7,3.0,3.268798,0.268798,0.072252,0.0,0.089599
7941,graph_powerlaw_cluster_graph_n7,3.0,2.921244,0.078756,0.006203,0.0,0.026252


In [9]:
df['RE'].sum()/len(df)

0.0631602579158256

In [10]:
def group_by_table(df, groupby):
    values_count = df[groupby].value_counts().reset_index()
    values_count = values_count.rename(columns={"count": "Count"})
    re_sum = df.groupby(groupby)["RE"].sum().reset_index()
    mre_error = df.groupby(groupby)["RE"].mean().reset_index()
    mre_error = mre_error.rename(columns={"RE": "MRE"})
    mse_error = df.groupby(groupby)["Difference_sq"].mean().reset_index()
    mse_error = mse_error.rename(columns={"Difference_sq": "MSE"})

    return values_count, re_sum, mre_error, mse_error

# Group by Rank

In [11]:
groupby = ['Rank']

values_count, re_sum, mre_error, mse_error = group_by_table(df, groupby)

df_result = pd.merge(values_count, re_sum).sort_values(by=['Count', 'RE'], ascending=[False, False])
df_result = pd.merge(df_result, mre_error)
df_result = pd.merge(df_result, mse_error)
df_result.to_csv("analysis_results/test_analysis_grp_by_rank.csv", index=False)
df_result


,Rank,Count,RE,MRE,MSE
0,0.0,2482,65.067671,0.026216,0.022123
1,3.0,1740,138.485213,0.079589,0.106860
2,2.0,1647,167.618802,0.101772,0.078168
3,1.0,900,46.536075,0.051707,0.004120
4,4.0,756,58.066993,0.076808,0.140168
5,5.0,151,13.354980,0.088444,0.249153
6,6.0,103,5.992254,0.058177,0.125566
7,7.0,66,4.044359,0.061278,0.185107
8,8.0,59,1.888574,0.032010,0.068810
9,9.0,24,0.035576,0.001482,0.000267


# Group by Dataset

In [12]:
groupby = ['Graph']

values_count, re_sum, mre_error, mse_error = group_by_table(df, groupby)

df_result = pd.merge(values_count, re_sum).sort_values(by=['Count', 'RE'], ascending=[False, False])
df_result = pd.merge(df_result, mre_error)
df_result = pd.merge(df_result, mse_error)
df_result.to_csv("analysis_results/test_analysis_grp_by_dataset.csv", index=False)
df_result


,Graph,Count,RE,MRE,MSE
0,graph_random_regular_graph_n7_d4,3907,293.831011,0.075206,0.080568
1,star_graph_n13,2663,92.853374,0.034868,0.042513
2,graph_powerlaw_cluster_graph_n7,1350,114.234583,0.084618,0.088865
3,star_graph_n7,23,0.762960,0.033172,0.042309


# Group by Dataset W/O Rank 0

In [13]:
groupby = ['Graph']

df_wo_0 = df[df['Rank']!=0]
values_count, re_sum, mre_error, mse_error = group_by_table(df_wo_0, groupby)

df_result = pd.merge(values_count, re_sum).sort_values(by=['Count', 'RE'], ascending=[False, False])
df_result = pd.merge(df_result, mre_error)
df_result = pd.merge(df_result, mse_error)
df_result.to_csv("analysis_results/test_analysis_grp_by_dataset_wo_rank0.csv", index=False)
df_result

,Graph,Count,RE,MRE,MSE
0,graph_random_regular_graph_n7_d4,3791,293.831011,0.077508,0.083033
1,graph_powerlaw_cluster_graph_n7,1241,114.234583,0.092050,0.096670
2,star_graph_n13,426,27.800519,0.065259,0.136861
3,star_graph_n7,3,0.748145,0.249382,0.324297


# Group by Dataset and Rank

In [14]:
groupby = ['Graph', 'Rank']

values_count, re_sum, mre_error, mse_error = group_by_table(df, groupby)

df_result = pd.merge(values_count, re_sum).sort_values(by=['Graph', 'Count'], ascending=[True, False])
df_result = pd.merge(df_result, mre_error)
df_result = pd.merge(df_result, mse_error)
df_result.to_csv("analysis_results/test_analysis_grp_by_dataset_rank.csv", index=False)
df_result

,Graph,Rank,Count,RE,MRE,MSE
0,graph_powerlaw_cluster_graph_n7,2.0,434,57.635023,0.132800,0.123359
1,graph_powerlaw_cluster_graph_n7,1.0,331,15.170192,0.045831,0.004349
2,graph_powerlaw_cluster_graph_n7,3.0,308,29.378788,0.095386,0.135499
3,graph_powerlaw_cluster_graph_n7,4.0,151,10.816889,0.071635,0.127075
4,graph_powerlaw_cluster_graph_n7,0.0,109,0.000000,0.000000,0.000000
5,graph_powerlaw_cluster_graph_n7,5.0,17,1.233690,0.072570,0.239301
6,graph_random_regular_graph_n7_d4,3.0,1411,105.260748,0.074600,0.097002
7,graph_random_regular_graph_n7_d4,2.0,1207,107.826207,0.089334,0.059603
8,graph_random_regular_graph_n7_d4,1.0,569,31.365883,0.055125,0.003986
9,graph_random_regular_graph_n7_d4,4.0,553,43.409301,0.078498,0.148205
